In [1]:
import os
import time
import csv
import numpy as np
import pandas as pd

import sklearn.metrics
import re
import unicodedata
from tqdm import tqdm
import sys
import hashlib
import random


Input CSV

In [2]:


import re
df = pd.read_csv("../crawled_issue/data_weather.csv")
df_unique = df.drop_duplicates(subset=["Issue ID"], keep='first')
df = df_unique
print(df.shape)


(34, 4)


In [3]:
# Find duplicate rows based on the 'Name' column
duplicate_rows = df.duplicated(subset=['Issue ID'])
number_of_duplicates = duplicate_rows.sum() - duplicate_rows.iloc[0]
print(number_of_duplicates)

0


In [4]:
# !pip3 install openpyxl 
excel_data = pd.read_excel('../dataset/Secret-Regular-Expression.xlsx')

# Read the values of the file in the dataframe
regex = pd.DataFrame(excel_data, columns=[
'Pattern_ID','Secret Type',	'Regular Expression','Source'])

In [5]:

data_dict={}
for j in df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df["Issue Body"][j])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        # file_path = "output.txt"

        # with open(file_path, 'w') as file:
        #                 file.write(cleaned_text)
        data_dict[j] = {'Issue ID':df['Issue ID'][j],'Issue Body':cleaned_text}
        # idx = idx+1
    


cleaned_text_data = pd.DataFrame.from_dict(data_dict, "index")
cleaned_text_data

,Issue ID,Issue Body
0,38,The application always calls when I try to sea...
1,37,missing MyObjectBox class and ItemHourlyDB_ cl...
2,33,close #32
3,32,"In some device like Xiaomi, set dark mode to t..."
4,30,close #29
5,29,Some libraries are out of date and should be u...
6,28,Hello sir can you please tell me where to put ...
7,27,When I open app it show keyboard like \r\n![si...
8,26,Can you tell me how to change the degree param...
9,25,"temperature is empty for four days , is it als..."


Candidate String Generation

In [6]:
###recovered
    
idx = 0
data_dict={}
# start = iter*100000
# end = (iter+1)*100000
for i in regex.index:
    #print(i,regex['Secret Type'][i]) #, regex['Regular Expression'][i])
    # if i%100==0:
    #     print("checkpoint")
    p = re.compile(regex['Regular Expression'][i])
    
    # print("=====================================================================")
    
    for j in df.index:
        
        cleaned_text = cleaned_text_data.loc[j, 'Issue Body']
            # Now you can use 'cleaned_text' for further processing
       
        matches = re.findall(p,cleaned_text)
        for match in set(matches):
                data_dict[idx] = {'Type': regex['Secret Type'][i], 'Issue ID':df['Issue ID'][j],'Candidate String':match} #,'Entropy':shannon_entropy(match)}
                idx = idx+1
    


data = pd.DataFrame.from_dict(data_dict, "index")
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
print(data.shape)
data.to_csv('../crawled_issue/issues-with-candidate-strings.csv')

C:\Users\ahmed\AppData\Local\Temp\ipykernel_23812\3600390447.py:11: FutureWarning: Possible nested set at position 35
  p = re.compile(regex['Regular Expression'][i])


(1, 3)


C:\Users\ahmed\AppData\Local\Temp\ipykernel_23812\3600390447.py:11: DeprecationWarning: Flags not at the start of the expression '/(?i)-----\\s*?BEGIN[' (truncated) but at position 1
  p = re.compile(regex['Regular Expression'][i])


In [7]:
data=pd.read_csv('../crawled_issue/issues-with-candidate-strings.csv')

In [8]:

data = data.rename(columns={'Issue ID': 'Issue_id'})
print(data.shape)
print(data.head())
merged_df = df.merge(data, left_on=['Issue ID'], right_on=['Issue_id'])
print(merged_df.shape)
columns_to_remove = ['Issue_id']
merged_df.drop(columns=columns_to_remove, inplace=True)
print(merged_df.columns)
# for i in range(merged_df.shape[0]):
#     merged_df["Issue Body"][i]=str(merged_df["Issue Body"][i])

(1, 4)
   Unnamed: 0             Type  Issue_id   Candidate String
0           0  Generic Pattern        27  2021-07-14-222921
(1, 8)
Index(['Issue ID', 'Title', 'Issue Body', 'Closed At', 'Unnamed: 0', 'Type',
       'Candidate String'],
      dtype='object')


In [9]:
# merged_df.to_csv('../crawled_issue/merged_issues-with-candidate-strings.csv')
# merged_df = pd.read_csv('../crawled_issue/merged_issues-with-candidate-strings.csv')

Context Window Generation

In [10]:
def create_context_window(text, target_string, window_size=125):

    target_index = text.find(target_string)
    #print(target_index)

    if target_index != -1:
        start_index = max(0, target_index - window_size)
        end_index = min(len(text), target_index + len(target_string) + window_size)
        context_window = text[start_index:end_index]
        return context_window

    return None

# Apply the create_context_window function to each row in the DataFrame
merged_df['modified_text'] = merged_df.apply(lambda row: create_context_window(row['Issue Body'], row['Candidate String']), axis=1)
print(merged_df.shape)
print(merged_df.head())

(1, 8)
   Issue ID          Title                                         Issue Body  \
0        27  Hide Keyboard  When I open app it show keyboard like \r\n![si...   

              Closed At  Unnamed: 0             Type   Candidate String  \
0  2021-09-15T22:41:37Z           0  Generic Pattern  2021-07-14-222921   

                                       modified_text  
0  When I open app it show keyboard like \r\n![si...  


In [11]:
count=0
inverse_count=0

for i in range(merged_df.shape[0]):
  #print(i)
  main_string=merged_df['Issue Body'][i]
  substring=merged_df['Candidate String'][i]
  #print(main_string.find(substring))
  if main_string.find(substring)!=-1:
    count+=1
  else:
    inverse_count+=1
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))

Mismatch: 0
Ok: 1


In [12]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

c:\Users\ahmed\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
X_issue_ids = merged_df['Issue ID'].tolist()
X_text_test = merged_df['Issue Body'].tolist()  # Convert the 'text' column to a list of strings
X_candidate_test = merged_df['Candidate String'].tolist()  # Convert the 'candidate_string' column to a list of strings

In [14]:
def encode_texts(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    return encodings

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [15]:
# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text_encodings, candidate_encodings, labels):
        self.text_encodings = text_encodings
        self.candidate_encodings = candidate_encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): #it works fine for training
        text_input_ids = self.text_encodings['input_ids'][idx]
        text_attention_mask = self.text_encodings['attention_mask'][idx]
        candidate_input_ids = self.candidate_encodings['input_ids'][idx]
        candidate_attention_mask = self.candidate_encodings['attention_mask'][idx]
        label = torch.tensor(self.labels[idx])

        return text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, label

In [16]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model_path = "../models/25k/adamW_cntxt125_data25k_pre.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode for inference

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [17]:
text_body_encodings_test = encode_texts(X_text_test)
candidate_encodings_test = encode_texts(X_candidate_test)

In [18]:
print(len(X_text_test))
Y_labels = [0] * len(X_text_test)
Y = np.array(Y_labels)
Y_ =Y.astype(int)
print(Y_)

1
[0]


In [19]:
test_dataset = CustomDataset(text_body_encodings_test, candidate_encodings_test, Y_)

In [20]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [21]:
model.to(device)
model.eval()  # Set the model in evaluation mode
c=0
predicted_labels_list = []
with torch.no_grad():

    for batch in test_loader:
        print("Batch %d"%c)
        c+=1

        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch

        # Move tensors to the device
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = (
            text_input_ids.to(device),
            text_attention_mask.to(device),
            candidate_input_ids.to(device),
            candidate_attention_mask.to(device),
            labels.to(device)
        )

        # Perform inference

        outputs = model(input_ids=text_input_ids.type(torch.LongTensor).cuda(), attention_mask=text_attention_mask.type(torch.LongTensor).cuda())
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        # print(f"predicted_labels: {predicted_labels}")
        predicted_labels_list.append(predicted_labels[0])

Batch 0


In [22]:
predicted_labels_list_output = [f.cpu().numpy().tolist() for f in predicted_labels_list]
print(predicted_labels_list_output)

[0]


Flagged CSV Generation

In [23]:

data_res={}
indx=0
for i in range(len(predicted_labels_list_output)):
    if predicted_labels_list_output[i] == 1 :
        data_res[indx]={"Issue ID":df[df['Issue ID'] == X_issue_ids[i]]['Issue ID'].values[0],"Issue Body":df[df['Issue ID'] == X_issue_ids[i]]['Issue Body'].values[0],"Candidate String":X_candidate_test[i],"RepoName":"weather"}
        indx=indx+1
print(len(data_res))
data = pd.DataFrame.from_dict(data_res, "index")
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
print(data.shape)
data.to_csv('../results/flagged_reallifeadamW125_11.csv')
        

0
(0, 0)


In [24]:

# secret_indexes = []
# for i in range(len(predicted_labels_list_output)):
#     if predicted_labels_list_output[i] == 1 :
#         secret_indexes.append(X_issue_ids[i])
# secret_indexes = list(set(secret_indexes))
# print(secret_indexes)
# print(len(secret_indexes))